# Hospital Staff Scheduling Optimization  
**Course:** Optimization and Analytics — Bachelor in Data Science and Engineering, UC3M  
**Students:** Fenz & Kahlkopf  
**Deadline:** November 16, 2025

---

# Introduction
In this project, we develop a **linear and mixed-integer optimization model** for the weekly scheduling of hospital staff.  
The goal is to assign doctors, nurses, and other healthcare workers to morning, afternoon, and night shifts while minimizing total staffing costs and respecting multiple operational and human constraints.

The dataset `hospital_staff.csv` contains information about 20 employees, including:
- Role, gender, and experience level  
- Maximum weekly working hours  
- Hourly cost  
- Individual availability and shift preferences  

This model helps hospital administrators create efficient and fair schedules that:
- Guarantee the required number of staff in every shift,  
- Respect employees’ availability and workload limits,  
- Avoid infeasible or unfair assignments,  
- Minimize the total labor cost.

In part (a), we formulate the **linear programming (LP)** version of the problem.  
In part (b), we implement and solve the model using realistic data.  
In part (c), we analyze **sensitivities (shadow prices)** to understand how changes in constraints affect costs.  
Finally, in part (d), we extend the model to a **mixed-integer programming (MIP)** version by adding logical and conditional constraints such as rest periods after night shifts and maximum weekly work limits.


## (a) Linear Optimization Model — General Formulation

### Objective
The goal is to assign hospital staff members (doctors, nurses, interns, technicians, and instructors)  
to morning, afternoon, and night shifts over a weekly horizon **while minimizing total labor cost** and  
respecting individual availability, workload, and staffing requirements.

---

### Sets
| Symbol | Description |
|:--------|:-------------|
| \( I \) | Set of all staff members \( i \in I \) (20 employees) |
| \( D \) | Set of days in the planning horizon (e.g., Monday–Sunday) |
| \( S \) | Set of shifts per day (Morning, Afternoon, Night) |

---

### Parameters
| Symbol | Description |
|:--------|:-------------|
| \( c_{i,s} \) | Cost of assigning staff \( i \) to shift \( s \) (€/hour) |
| \( h_s \) | Duration of shift \( s \) (hours) |
| \( a_{i,s} \in \{0,1\} \) | Availability of staff \( i \) for shift \( s \) (1 = available, 0 = not available) |
| \( H_i \) | Maximum working hours per week for staff \( i \) |
| \( R_{s,d} \) | Minimum number of required staff members for shift \( s \) on day \( d \) |

---

### Decision Variables
| Symbol | Description |
|:--------|:-------------|
| \( x_{i,s,d} \in [0,1] \) | 1 if staff member \( i \) works on day \( d \) in shift \( s \); 0 otherwise |

In the **LP model**, \( x_{i,s,d} \) can take continuous values in [0,1].  
In the **MIP extension**, it will become binary (\( x_{i,s,d} \in \{0,1\} \)).

---

### Objective Function
Minimize total labor cost over all staff, shifts, and days:

$$
\text{Minimize } Z = \sum_{i \in I} \sum_{s \in S} \sum_{d \in D} c_{i,s} \, h_s \, x_{i,s,d}
$$

---

### Constraints

1. **Shift coverage requirement:**  
   Each shift on each day must be covered by the required number of staff:
   $$
   \sum_{i \in I} x_{i,s,d} \ge R_{s,d} \quad \forall s \in S, \, d \in D
   $$

2. **At most one shift per person per day:**  
   A staff member can only work one shift per day:
   $$
   \sum_{s \in S} x_{i,s,d} \le 1 \quad \forall i \in I, \, d \in D
   $$

3. **Weekly working hour limit:**  
   Each staff member cannot exceed their maximum weekly hours:
   $$
   \sum_{s \in S} \sum_{d \in D} h_s \, x_{i,s,d} \le H_i \quad \forall i \in I
   $$

4. **Availability constraint:**  
   Staff can only be assigned to shifts they are available for:
   $$
   x_{i,s,d} \le a_{i,s} \quad \forall i \in I, \, s \in S, \, d \in D
   $$

5. **Non-negativity:**  
   $$
   x_{i,s,d} \ge 0
   $$

---

### Model Summary
This linear programming formulation captures a realistic hospital scheduling scenario where the hospital must:
- Ensure adequate coverage per shift and day,  
- Respect each employee’s workload and availability,  
- Minimize total operational cost.

In [32]:
import pandas as pd
import pyomo.environ as pyo

staff = pd.read_csv("hospital_staff.csv")
staff.head()


,staff_id,name,gender,role,experience_level,max_hours_per_week,cost_per_hour,shift_preference,availability_morning,availability_afternoon,availability_night
0,1,Alice Johnson,Male,Intern,Mid,27,11,Morning,1,1,0
1,2,Bob Smith,Male,Technician,Mid,30,18,Morning,1,1,0
2,3,Clara Lee,Male,Nurse,Mid,45,28,Afternoon,0,0,1
3,4,David Brown,Female,Nurse,Senior,35,22,Night,1,1,1
4,5,Eva Green,Female,Nurse,Junior,40,21,Morning,0,1,1


In [33]:
# --- Sets ---
DAYS = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
SHIFTS = ["Morning", "Afternoon", "Night"]
I = list(staff["staff_id"].astype(int))

# --- Parameters ---
# Shift duration (hours)
h = {"Morning": 8, "Afternoon": 8, "Night": 10}

# Minimum required staff per shift/day
R = {(s,d): (5 if s in ["Morning","Afternoon"] else 2) for s in SHIFTS for d in DAYS}

# Availability a_{i,s}
A = {}
for _, r in staff.iterrows():
    A[int(r.staff_id)] = {
        "Morning": int(r.availability_morning),
        "Afternoon": int(r.availability_afternoon),
        "Night": int(r.availability_night),
    }

# Cost c_{i,s} = cost_per_hour * shift_length (+ small penalty if not preferred)
c = {}
for _, r in staff.iterrows():
    for s in SHIFTS:
        base_cost = float(r.cost_per_hour) * h[s]
        if r.shift_preference != s:
            base_cost += 2.0
        c[(int(r.staff_id), s)] = base_cost

# Weekly hour limit
H = {int(r.staff_id): int(r.max_hours_per_week) for _, r in staff.iterrows()}



In [34]:

# --- Create Pyomo model ---
model = pyo.ConcreteModel()

# Decision variables x[i,s,d] ∈ [0,1]
model.x = pyo.Var(I, SHIFTS, DAYS, domain=pyo.NonNegativeReals, bounds=(0, 1))

# Objective: Minimize total cost
def obj_rule(model):
    return sum(c[(i, s)] * model.x[i, s, d] for i in I for s in SHIFTS for d in DAYS)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# --- Constraints ---
model.constraints = pyo.ConstraintList()

# (1) Shift coverage: each shift/day must meet requirement
for s in SHIFTS:
    for d in DAYS:
        model.constraints.add(sum(model.x[i, s, d] for i in I) >= R[(s, d)])

# (2) One shift per person per day
for i in I:
    for d in DAYS:
        model.constraints.add(sum(model.x[i, s, d] for s in SHIFTS) <= 1)

# (3) Weekly hour limit
for i in I:
    model.constraints.add(sum(h[s] * model.x[i, s, d] for s in SHIFTS for d in DAYS) <= H[i])

# (4) Availability
for i in I:
    for s in SHIFTS:
        for d in DAYS:
            model.constraints.add(model.x[i, s, d] <= A[i][s])

# (5) Minimum working hours per person (optional fairness constraint)
L = {}  # lower hour bounds
for _, r in staff.iterrows():
    role = r["role"]
    if role == "Doctor":
        L[int(r.staff_id)] = 15
    elif role == "Nurse":
        L[int(r.staff_id)] = 15
    elif role == "Technician":
        L[int(r.staff_id)] = 10
    elif role == "Instructor":
        L[int(r.staff_id)] = 10
    else:  # Intern
        L[int(r.staff_id)] = 5

for i in I:
    model.constraints.add(sum(h[s] * model.x[i, s, d] for s in SHIFTS for d in DAYS) >= L[i])


# --- Solve model ---
solver = pyo.SolverFactory('glpk')
result = solver.solve(model, tee=False)

print(f"Solver status: {result.solver.status}")
print(f"Termination condition: {result.solver.termination_condition}")
print(f"Objective (Total cost): {pyo.value(model.obj):.2f}")


Solver status: ok
Termination condition: optimal
Objective (Total cost): 15759.75


In [35]:
# --- Result analysis ---
assignments = []
for i in I:
    name = staff.loc[staff.staff_id == i, "name"].values[0]
    role = staff.loc[staff.staff_id == i, "role"].values[0]
    total_hours = sum(h[s] * pyo.value(model.x[i, s, d]) for s in SHIFTS for d in DAYS)
    assignments.append({
        "ID": i,
        "Name": name,
        "Role": role,
        "Total_hours": round(total_hours, 2),
        "Max_hours": H[i],
    })

df_hours = pd.DataFrame(assignments).sort_values("Total_hours", ascending=False)
display(df_hours.head(10))

# Check coverage
coverage = []
for s in SHIFTS:
    for d in DAYS:
        assigned = sum(pyo.value(model.x[i, s, d]) for i in I)
        coverage.append({
            "Shift": s,
            "Day": d,
            "Required": R[(s, d)],
            "Assigned": round(assigned, 2)
        })
df_coverage = pd.DataFrame(coverage)
display(df_coverage)

# Full week schedule
schedule = []

for d in DAYS:
    for s in SHIFTS:
        assigned = []
        for i in I:
            val = pyo.value(model.x[i, s, d], exception=False)
            if val and val > 0.01:  # tolerance: ignore 0 or fractional noise
                name = staff.loc[staff.staff_id == i, "name"].values[0]
                assigned.append(name)
        schedule.append({
            "Day": d,
            "Shift": s,
            "Assigned_Staff": ", ".join(assigned) if assigned else "—",
            "Required": R[(s, d)],
            "Assigned_Count": len(assigned)
        })

df_schedule = pd.DataFrame(schedule)

# Sort days in correct order
day_order = {d: i for i, d in enumerate(DAYS)}
df_schedule["Day_Order"] = df_schedule["Day"].map(day_order)
df_schedule = df_schedule.sort_values(["Day_Order", "Shift"]).drop(columns="Day_Order")

display(df_schedule)


,ID,Name,Role,Total_hours,Max_hours
11,12,Liam Hall,Nurse,45.0,45
5,6,Frank Harris,Nurse,44.0,44
13,14,Noah Scott,Nurse,44.0,44
18,19,Samuel Evans,Nurse,43.0,43
9,10,Jack Lewis,Nurse,40.0,40
4,5,Eva Green,Nurse,40.0,40
2,3,Clara Lee,Nurse,40.0,45
8,9,Isabel Clark,Nurse,38.0,38
10,11,Karen Walker,Nurse,37.0,37
6,7,Grace Wilson,Technician,36.0,36


,Shift,Day,Required,Assigned
0,Morning,Mon,5,5.0
1,Morning,Tue,5,5.0
2,Morning,Wed,5,5.0
3,Morning,Thu,5,5.0
4,Morning,Fri,5,5.0
5,Morning,Sat,5,5.0
6,Morning,Sun,5,5.0
7,Afternoon,Mon,5,5.0
8,Afternoon,Tue,5,5.0
9,Afternoon,Wed,5,5.0


,Day,Shift,Assigned_Staff,Required,Assigned_Count
1,Mon,Afternoon,"David Brown, Isabel Clark, Paul Turner, Rachel...",5,6
0,Mon,Morning,"Alice Johnson, Bob Smith, Frank Harris, Grace ...",5,6
2,Mon,Night,"Clara Lee, David Brown, Henry Young, Karen Walker",2,4
4,Tue,Afternoon,"Eva Green, Isabel Clark, Olivia Adams, Samuel ...",5,5
3,Tue,Morning,"Bob Smith, Grace Wilson, Liam Hall, Noah Scott...",5,5
5,Tue,Night,"Clara Lee, Jack Lewis, Karen Walker",2,3
7,Wed,Afternoon,"Eva Green, Isabel Clark, Olivia Adams, Rachel ...",5,6
6,Wed,Morning,"Bob Smith, Frank Harris, Grace Wilson, Liam Ha...",5,6
8,Wed,Night,"Henry Young, Karen Walker",2,2
10,Thu,Afternoon,"Eva Green, Grace Wilson, Jack Lewis, Mia Allen...",5,7


### Interpretation – Weekly Staff Schedule (Part b continuation)

The following table represents the **optimized hospital schedule** for one week, 
showing which employees are assigned to each day and shift.  
Each shift meets the required staffing level while respecting all constraints defined in the model.

#### Key observations:

- **Full coverage:**  
  Every shift (morning, afternoon, and night) has at least the required number of staff members, 
  satisfying the coverage constraint \( \sum_i x_{i,s,d} \ge R_{s,d} \).

- **Feasibility:**  
  No one exceeds their weekly hour limit \( H_i \), and all assignments respect the 
  availability matrix \( A_{i,s} \). This ensures that staff are only scheduled for 
  shifts they are available for.

- **Cost efficiency:**  
  The model minimizes total labor cost (€15,667), balancing cheaper and more available 
  staff across the week. Less expensive employees are prioritized, 
  while others are used to fill specific availability gaps.

- **Fair workload distribution:**  
  Most employees work between **35–45 hours per week**, which fits within typical 
  hospital workload expectations. Some staff members work fewer hours because 
  they are only partially available (e.g., limited night shift availability).

- **Practical realism:**  
  The resulting plan is consistent with real hospital operations: nurses dominate 
  morning and afternoon shifts, while a smaller team covers night shifts. 
  The optimization model automatically assigns staff efficiently while maintaining 
  realistic coverage and fairness.

Overall, the schedule confirms that the **linear optimization model** successfully 
balances staffing needs, employee constraints, and operational cost — achieving an 
optimal and realistic weekly duty plan.
